<a href="https://colab.research.google.com/github/dalton-rutledge/Tweeter/blob/master/Convert_h5_to_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import tf and keras
import tensorflow as tf
keras = tf.keras

In [0]:
#loading keras model
MyModel = keras.models.load_model("/content/drive/My Drive/Tweeter/FinalKerasTEST.h5") 
MyModel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               10752     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
_________________________________________________________________
activation_2 (Activation)    (None, 10)               

In [0]:
#test to see that the loaded model works properly
import numpy as np
#feeding sample mfccs for class 0.
sample_mfccs = [-181.5716300186619, 13.067425309834757, -110.85055134173761, -9.068524171220472, -1.2984628746989908,
                39.436294413594176, -11.955572530494793, -8.27045641645622, -1.6398283985684026, 7.107573167031457,
                -14.903284713167196, -1.2327483320208874, 6.1677468305135825, 6.366361888102694, 2.250384814234832,
                13.440506878383358, -5.911123458542153, 1.1202938047476207, -1.895890046440057, -2.0409995023706275]
Xs = np.array(sample_mfccs, dtype=np.float32).reshape(1,20)
print(Xs.shape)
MyModel.predict(Xs)

(1, 20)


array([[1.0000000e+00, 9.7003078e-19, 2.2786080e-11, 7.1497522e-18,
        3.1314914e-24, 7.4506187e-15, 6.3757052e-20, 2.3781006e-16,
        6.0799132e-27, 6.9530119e-22]], dtype=float32)

In [0]:
#convert the .h5 model to .tflite and save 
converter = tf.lite.TFLiteConverter.from_keras_model(MyModel)
tflite_model = converter.convert()

file = open( '/content/drive/My Drive/Tweeter/Models/tweeterpredictor.tflite' , 'wb' ) 
file.write( tflite_model )


1115572

In [0]:
#test the .tflite model:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/drive/My Drive/Tweeter/Models/tweeterpredictor.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data with input_data. Test model on mfccs extracted from class 0 with input_data2
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
input_data2 = Xs

interpreter.set_tensor(input_details[0]['index'], input_data2)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[1.0000000e+00 9.7003450e-19 2.2786124e-11 7.1497795e-18 3.1314794e-24
  7.4506466e-15 6.3757292e-20 2.3781096e-16 6.0799132e-27 6.9530119e-22]]
